In [11]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import matplotlib.pyplot as plt
import panel as pn
import os
import base64

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

12월 일사량, 온도 시간별 누적 평균 그래프 시각화

In [8]:
#12월 일사량 누적, 온도, 풍속 평균 그래프 시각화
# --------------------------
# 1. EPW 파일 불러오기
# --------------------------
epw_path = "C:\Users\USER\Desktop\#캡스톤\#패널\data\KOR_KG_Seoul-Seongnam.AP.471110_TMYx.2004-2018\KOR_KG_Seoul-Seongnam.AP.471110_TMYx.2004-2018.epw"  # <- 여기에 EPW 파일 경로 입력
df = pd.read_csv(epw_path, skiprows=8, header=None)

# --------------------------
# 2. 날짜 및 시간 처리
# --------------------------
df['Month'] = df[1]
df['Day'] = df[2]
df['Hour'] = df[3] - 1  # EPW의 시간은 1~24로 표기 → 0~23으로 보정

# 날짜 컬럼 추가 (선택적, 타임스탬프 만들기용)
df['Datetime'] = pd.to_datetime({
    'year': 2023,
    'month': df['Month'],
    'day': df['Day'],
    'hour': df['Hour']
}, errors='coerce')

In [10]:
# 1. 주요 기상 변수 추출
df['GHI_kWh'] = df[13] / 1000
df['Temp'] = df[6]

# 2. 데이터 필터링 (2010년 12월 데이터)
dec_2010 = df[(df[0] == 2010) & (df['Month'] == 12)].copy()

# 3. 시간 필터링 (06시~18시 사용)
hourly_avg_filtered = dec_2010[(dec_2010['Hour'] >= 6) & (dec_2010['Hour'] <= 18)].groupby('Hour')[['GHI_kWh', 'Temp']].mean()

# 4. X축 Label
hour_labels = [f"{hour:02d}:00" for hour in hourly_avg_filtered.index]

# 5. 그래프 시각화
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(specs=[[{"secondary_y": True}]])

# Bar plot (Radiation)
fig.add_trace(
    go.Bar(
        x=hour_labels,
        y=hourly_avg_filtered['GHI_kWh'],
        name='Radiation (kWh/m²)',
        marker_color='gray',
        opacity=0.8  # Bar 투명도 추가
    ),
    secondary_y=False,
)

# Temperature (Temp → 점+선)
fig.add_trace(
    go.Scatter(
        x=hour_labels,
        y=hourly_avg_filtered['Temp'],
        name='Temperature (°C)',
        mode='lines+markers',
        line=dict(color='skyblue', width=3)  # 라인 두께 3로 설정
    ),
    secondary_y=True,
)

# Layout 설정
fig.update_layout(
    legend=dict(
        x=0.85,
        y=0.98,
        xanchor='left',
        yanchor='top',
        font=dict(family="Noto Sans KR Medium", size=20, color="black"),
        bgcolor="rgba(255,255,255,0.7)",
        bordercolor="black",
        borderwidth=0
    ),
    margin=dict(l=80, r=80, t=120, b=120),
    plot_bgcolor='white',
    paper_bgcolor='whitesmoke'
)

# X축 설정
fig.update_xaxes(
    title_text="Time",
    type='category',
    tickangle=0,
    showgrid=True,
    title_font=dict(size=20, family="Noto Sans KR Medium", color="black"),
    tickfont=dict(size=18, family="Noto Sans KR Medium", color="black")  # 글자 크기 조절 추가
)

# Y축 Radiation
fig.update_yaxes(
    title_text="Radiation (kWh/m²)",
    title_font=dict(size=20, family="Noto Sans KR Medium", color="black"),
    tickfont=dict(size=15, family="Noto Sans KR Medium", color="black"),  # 여기 추가!
    secondary_y=False,
    range=[0, 0.5],
    showgrid=True,
    gridcolor='rgba(0,0,0,0.5)',
    gridwidth=1,
    tickvals=[round(x * 0.05, 8) for x in range(11)],
    ticktext=[f"{round(x * 0.05, 2):.2f}" for x in range(11)],
    tickmode='array'
)

# Y축 Temp → grid 표시 안함
fig.update_yaxes(
    title_text="Temperature (°C)",
    title_font=dict(size=20, family="Noto Sans KR Medium", color="black"),
    tickfont=dict(size=15, family="Noto Sans KR Medium", color="black"),  # 여기 추가!
    secondary_y=True,
    range=[-4, 2],
    showgrid=False
)

# Show the plot
fig.show()

# Save as HTML
#fig.write_html("Data Analysis.html")


12월 라이노 일사량 분석 슬라이더

In [13]:
pn.extension()

# ① 폴더 경로
folder_path = r"C:\Users\USER\Desktop\#캡스톤\#패널\radiation analysis"

# ② 총 프레임 수
num_frames = 372

# ③ base64 변환 함수
def load_base64_png(file_path):
    with open(file_path, "rb") as f:
        data = f.read()
    encoded = base64.b64encode(data).decode("utf-8")
    return f"data:image/png;base64,{encoded}"

# ④ base64 이미지 리스트 만들기
frame_images = []
for i in range(1, num_frames + 1):
    file_path = os.path.join(folder_path, f"frame ({i}).png")  # 공백 주의 → 실제 파일명 확인 필요
    if not os.path.exists(file_path):
        print(f"Missing file: {file_path}")
    else:
        frame_images.append(load_base64_png(file_path))

# ⑤ 초기 이미지 Pane → HTML 사용
img_pane = pn.pane.HTML(f'<img src="{frame_images[0]}" width="800">')

# ⑥ 슬라이더
current_frame = pn.widgets.IntSlider(name='Frame', start=1, end=len(frame_images), step=1)

# ⑦ Play 상태 관리
playing = [False]

# ⑧ 이미지 업데이트 함수
def update_image(event):
    frame_idx = event.new
    img_pane.object = f'<img src="{frame_images[frame_idx - 1]}" width="800">'

current_frame.param.watch(update_image, 'value')

# ⑨ Prev button
def prev_callback(event):
    if current_frame.value > 1:
        current_frame.value -= 1

# ⑩ Next button
def next_callback(event):
    if current_frame.value < len(frame_images):
        current_frame.value += 1

# ⑪ Play/Pause button
def animate():
    if playing[0]:
        if current_frame.value < len(frame_images):
            current_frame.value += 1
        else:
            current_frame.value = 1

prev_button = pn.widgets.Button(name='⏮ Prev', button_type='primary')
prev_button.on_click(prev_callback)

next_button = pn.widgets.Button(name='Next ⏭', button_type='primary')
next_button.on_click(next_callback)

play_button = pn.widgets.Toggle(name='▶ Play / ⏸ Pause', button_type='success')

cb = pn.state.add_periodic_callback(animate, period=500)    # period: frmae당 시간
cb.stop()

def play_callback(event):
    if event.new:
        playing[0] = True
        cb.start()
    else:
        playing[0] = False
        cb.stop()

play_button.param.watch(play_callback, 'value')

# ⑫ 레이아웃 구성
controls = pn.Row(prev_button, play_button, next_button)
layout = pn.Column(current_frame, controls, img_pane)

# ⑬ HTML 저장 → base64라서 HTML 파일 하나로 동작 가능!
#layout.save("animation_buttons_panel_base64_FINAL.html", embed=True)

# ⑭ Panel serve로 확인도 가능
layout.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'602267f5-4c5f-47a3-8ca3-0ab4516d5bdd': {'version…

In [9]:
import dash
from dash import html, dcc
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# ──────────────────────────────────────
# 1️⃣ EPW 기반 그래프 데이터 준비
# ──────────────────────────────────────

# 예시 DataFrame 구성 (여기에 본문 df 넣으시면 됩니다)
# 지금은 코드에서 dec_2010의 결과 → hourly_avg_filtered 사용

# 예시 복사 (실제 코드에서는 hourly_avg_filtered 그대로 사용 가능)
hour_labels = ['06:00', '07:00', '08:00', '09:00', '10:00', '11:00',
               '12:00', '13:00', '14:00', '15:00', '16:00', '17:00', '18:00']

# 예시 값 → 실제로 hourly_avg_filtered['GHI_kWh'], hourly_avg_filtered['Temp'] 넣으면 됩니다
ghi_values = [0.0, 0.02, 0.05, 0.08, 0.12, 0.18, 0.22, 0.20, 0.15, 0.10, 0.05, 0.02, 0.0]
temp_values = [-5, -4, -3, -1, 0, 2, 3, 2, 1, 0, -1, -3, -4]

# Plotly 그래프 구성
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Bar(
        x=hour_labels,
        y=ghi_values,
        name='Radiation (kWh/m²)',
        marker_color='gray',
        opacity=0.8
    ),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(
        x=hour_labels,
        y=temp_values,
        name='Temperature (°C)',
        mode='lines+markers',
        line=dict(color='skyblue', width=3)
    ),
    secondary_y=True,
)

fig.update_layout(
    title="12월 시간대별 Radiation & Temperature",
    legend=dict(x=0.85, y=0.98, bgcolor="rgba(255,255,255,0.7)"),
    margin=dict(l=80, r=80, t=120, b=120),
    plot_bgcolor='white',
    paper_bgcolor='whitesmoke'
)

fig.update_xaxes(
    title_text="Time",
    type='category',
    tickangle=0,
    showgrid=True,
    title_font=dict(size=20),
    tickfont=dict(size=18)
)

fig.update_yaxes(
    title_text="Radiation (kWh/m²)",
    secondary_y=False,
    range=[0, 0.5],
    showgrid=True,
    gridcolor='rgba(0,0,0,0.5)',
    gridwidth=1,
    tickvals=[round(x * 0.05, 8) for x in range(11)],
    ticktext=[f"{round(x * 0.05, 2):.2f}" for x in range(11)],
    tickmode='array'
)

fig.update_yaxes(
    title_text="Temperature (°C)",
    secondary_y=True,
    range=[-6, 4],
    showgrid=False
)

# ──────────────────────────────────────
# 2️⃣ Dash App 구성
# ──────────────────────────────────────

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("디지털트윈을 활용한 빙판길 사고 예방 (복정동 골목 분석)", style={'textAlign': 'center'}),

    # 카드 지표 영역 (예시 값)
    html.Div([
        html.Div("최대 경사도: 12°", style={'padding': '10px', 'border': '1px solid black'}),
        html.Div("평균 일사량: 120 kWh/m²", style={'padding': '10px', 'border': '1px solid black'}),
        html.Div("최저 기온: -5°C", style={'padding': '10px', 'border': '1px solid black'}),
        html.Div("위험지역 수: 5개", style={'padding': '10px', 'border': '1px solid black'}),
    ], style={'display': 'flex', 'gap': '10px', 'margin-bottom': '20px', 'justify-content': 'center'}),

    # Plotly 그래프
    dcc.Graph(figure=fig),

    # Animation Iframe
    html.H3("위험지역 누적 일사량 시각화 (애니메이션)", style={'textAlign': 'center'}),
    html.Iframe(src='animation_buttons_panel_base64_FINAL.html', width='100%', height='700px')
])

# ──────────────────────────────────────
# 3️⃣ App 실행
# ──────────────────────────────────────

if __name__ == '__main__':
    app.run(debug=True)


# 📊 Panel 핵심 메서드 정리

Panel(`pn`)은 Python에서 **대시보드, 웹 앱, 시각화 GUI**를 쉽게 만들 수 있게 해주는 라이브러리입니다. 아래는 실습에서 사용한 주요 메서드와 컴포넌트입니다.

---

## 🔹 `pn.pane.Markdown`

> **설명**: 텍스트를 Markdown 형식으로 보여주는 패널입니다. 제목, 설명, 강조 등에 사용됩니다.

In [ ]:
pn.pane.Markdown("### AAPL Stock Overview \n Text Test")

BokehModel(combine_events=True, render_bundle={'docs_json': {'66ef5ccb-636a-4ff3-a8a6-7c2a2e5425cd': {'version…

In [ ]:
description = """
        ### Microsoft Corporation (MSFT)
        
        Microsoft Corporation is an American multinational technology corporation that produces 
        computer software, consumer electronics, personal computers, and related services. 
        The company has shown strong growth in cloud services in recent years.
        
        The chart below shows the simulated stock price movement throughout 2024.
        """
    
description_pane = pn.pane.Markdown(description)
description_pane

ModuleNotFoundError: No module named 'jupyter_bokeh'

Markdown(str)

# [2] Panel을 이용한 대시보드 구성하기

In [10]:
# AAPL 시각화
aapl_stats = pd.DataFrame({
    'Metric': ['Current', 'High', 'Low', 'Average'],
    'Value': [
        f"${dfs['AAPL']['Close'].iloc[-1]:.2f}",
        f"${dfs['AAPL']['Close'].max():.2f}",
        f"${dfs['AAPL']['Close'].min():.2f}",
        f"${dfs['AAPL']['Close'].mean():.2f}"
    ]
})

aapl_plot = dfs['AAPL'].hvplot.line(
    y='Close',
    title='AAPL Stock Price',
    xlabel='Date',
    ylabel='Price (USD)',
    line_width=3,
    color='#1f77b4',
    height=400,
    shared_axes=False
)

aapl_panel = pn.Column(
    pn.pane.Markdown('### AAPL Stock Overview'),
    aapl_plot,
    pn.widgets.DataFrame(aapl_stats, name='AAPL Stats', height=150),
    css_classes=['stock-panel']
)

# MSFT 시각화
msft_stats = pd.DataFrame({
    'Metric': ['Current', 'High', 'Low', 'Average'],
    'Value': [
        f"${dfs['MSFT']['Close'].iloc[-1]:.2f}",
        f"${dfs['MSFT']['Close'].max():.2f}",
        f"${dfs['MSFT']['Close'].min():.2f}",
        f"${dfs['MSFT']['Close'].mean():.2f}"
    ]
})

msft_plot = dfs['MSFT'].hvplot.line(
    y='Close',
    title='MSFT Stock Price',
    xlabel='Date',
    ylabel='Price (USD)',
    line_width=3,
    color='#ff7f0e',
    height=400,
    shared_axes=False
)

msft_panel = pn.Column(
    pn.pane.Markdown('### MSFT Stock Overview'),
    msft_plot,
    pn.widgets.DataFrame(msft_stats, name='MSFT Stats', height=150),
    css_classes=['stock-panel']
)

# 대시보드 조립
panel1 = pn.Row(aapl_panel, msft_panel)

dashboard = pn.Column(
    pn.pane.Markdown('# Tech Stocks Dashboard - 2024 \n ### Visualization of Apple & Microsoft stock in 2024'),
    panel1,
    css_classes=['dashboard-container']
)

# [6] HTML로 저장 (선택)
dashboard.save('panel_dashboard_practice.html', embed=True)

# [3] 응용

In [1]:
import panel as pn
import pandas as pd
import numpy as np
import hvplot.pandas
import holoviews as hv
from datetime import datetime

# Panel을 초기화하고 테마 설정
pn.extension()
hv.extension('bokeh')

# 페이지 전체 너비 설정 - 16:9 모니터에 최적화
PAGE_WIDTH = 1600  # 일반적인 16:9 모니터 너비에 맞춤
CONTENT_WIDTH = 1500  # 내용물 너비

# 샘플 데이터 생성 함수 정의
def generate_sample_data():
    # 2024년 전체 날짜 생성
    dates = pd.date_range(start='2024-01-01', end='2024-12-31', freq='D')
    np.random.seed(42)  # 재현 가능성을 위한 랜덤 시드 설정
    
    # 3개 주식에 대한 랜덤 가격 생성 (값의 범위를 적절하게 조정)
    data = {
        'AAPL': 150 + np.random.normal(0, 3, len(dates)).cumsum(), 
        'MSFT': 250 + np.random.normal(0, 2, len(dates)).cumsum(),
        'GOOGL': 180 + np.random.normal(0, 2, len(dates)).cumsum()
    }
    
    # 각 주식에 대한 데이터프레임 생성
    dfs = {}
    for symbol, prices in data.items():
        df = pd.DataFrame({'Close': prices}, index=dates)
        dfs[symbol] = df
    return dfs

# 데이터 생성 및 주식 심볼 리스트 정의
symbols = ['AAPL', 'MSFT', 'GOOGL']
dfs = generate_sample_data()

# 페이지 1에 표시할 주식 (애플과 마이크로소프트)
page1_symbols = ['AAPL', 'MSFT']
# 페이지 2에 표시할 주식 (구글)
page2_symbols = ['GOOGL']

# CSS 스타일 정의 - 전체 레이아웃에 적용할 스타일
### 웹페이지에서 레이아웃과 시각적 스타일을 설정하는 CSS(Cascading Style Sheets) 코드입니다. Panel을 사용할 때 HTML 스타일을 커스터마이징하고 싶을 때 이렇게 직접 CSS를 정의해서 적용
css = """
body {
    margin: 0;
    padding: 0;
    width: 100%;
    height: 100%;
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    background-color: #f5f5f5;
}
.dashboard-container {
    width: 100%;
    max-width: """+str(PAGE_WIDTH)+"""px;
    margin: 0 auto;
    padding: 20px;
    box-sizing: border-box;
}
.dashboard-header {
    background-color: #ffffff;
    padding: 20px;
    border-radius: 10px;
    box-shadow: 0 2px 5px rgba(0,0,0,0.1);
    margin-bottom: 20px;
}
.dashboard-content {
    background-color: #ffffff;
    padding: 20px;
    border-radius: 10px;
    box-shadow: 0 2px 5px rgba(0,0,0,0.1);
}
.stock-panel {
    background-color: #f9f9f9;
    border-radius: 8px;
    padding: 15px;
    margin-bottom: 15px;
}
"""

# CSS 스타일 삽입
pn.extension(raw_css=[css])

# 대시보드의 제목 생성 (첫 페이지)
header1 = pn.pane.Markdown("# Stock Data Dashboard - Page 1\nVisualization of Apple and Microsoft stocks in 2024", align='center')

# 차트의 너비 계산 - 1행 2열에 맞게 설정
chart_width = int(CONTENT_WIDTH / 2 - 50)  # 여백 고려

# 페이지 1: 애플과 마이크로소프트 주식 패널 생성
page1_stock_panels = []

for symbol in page1_symbols:
    df = dfs[symbol]
    
    # 설명 텍스트 생성
    if symbol == 'AAPL':
        description = """
        ### Apple Inc. (AAPL)
        
        Apple Inc. is an American multinational technology company that designs, develops, 
        and sells consumer electronics, computer software, and online services. The company's 
        stock is known for its growth potential and is closely watched by investors worldwide.
        
        The chart below shows the simulated stock price movement throughout 2024.
        """
    else:  # MSFT
        description = """
        ### Microsoft Corporation (MSFT)
        
        Microsoft Corporation is an American multinational technology corporation that produces 
        computer software, consumer electronics, personal computers, and related services. 
        The company has shown strong growth in cloud services in recent years.
        
        The chart below shows the simulated stock price movement throughout 2024.
        """
    
    description_pane = pn.pane.Markdown(description, width=chart_width)
    
    # 차트 생성 - 너비를 모니터 크기에 맞게 조정
    plot = df.hvplot.line(
        y='Close',
        title=f'{symbol} Stock Price',
        xlabel='Date',
        ylabel='Price (USD)',
        height=400,
        width=chart_width,
        line_width=3,
        color='#1f77b4' if symbol == 'AAPL' else '#ff7f0e',
        shared_axes=False
    )
    
    # 통계 계산
    current = f"${df['Close'].iloc[-1]:.2f}"
    high = f"${df['Close'].max():.2f}"
    low = f"${df['Close'].min():.2f}"
    average = f"${df['Close'].mean():.2f}"
    
    # 통계 테이블 생성
    stats_df = pd.DataFrame({
        'Metric': ['Current', 'High', 'Low', 'Average'],
        'Value': [current, high, low, average]
    })
    stats_table = pn.widgets.DataFrame(stats_df, name=f'{symbol} Statistics', height=150, width=chart_width)
    
    # 주식별 패널 구성 - CSS 클래스 적용
    stock_panel = pn.Column(
        description_pane,
        plot,
        stats_table,
        css_classes=['stock-panel'],
        width=chart_width
    )
    page1_stock_panels.append(stock_panel)

# 페이지 2: 구글 주식 패널 생성
header2 = pn.pane.Markdown("# Stock Data Dashboard - Page 2\nVisualization of Google stock in 2024", align='center')

page2_stock_panels = []

for symbol in page2_symbols:
    df = dfs[symbol]
    
    # 구글 주식에 대한 설명 추가
    description = """
    ### Alphabet Inc. (GOOGL)
    
    Alphabet Inc. is an American multinational technology conglomerate holding company headquartered 
    in Mountain View, California. It was created through a restructuring of Google in 2015, and became 
    the parent company of Google and several former Google subsidiaries.
    
    The company is known for its dominance in the online advertising and search engine markets, as well 
    as its investments in areas including cloud computing, artificial intelligence, and autonomous vehicles.
    
    The chart below shows the simulated stock price movement throughout 2024.
    """
    
    description_pane = pn.pane.Markdown(description, width=CONTENT_WIDTH-100)
    
    # 구글 차트 생성 - 전체 너비 사용
    plot = df.hvplot.line(
        y='Close',
        title=f'{symbol} Stock Price',
        xlabel='Date',
        ylabel='Price (USD)',
        height=500,
        width=CONTENT_WIDTH-100,
        line_width=3,
        color='#2ca02c'
    )
    
    # 월별 변화율 계산
    monthly_df = df.resample('ME').last()  # 'ME'는 Month End를 의미
    monthly_df['MonthlyChange'] = monthly_df['Close'].pct_change() * 100
    
    # 월별 변화율 차트 - 전체 너비 사용
    monthly_plot = monthly_df.hvplot.bar(
        y='MonthlyChange',
        title='Monthly Change (%)',
        xlabel='Month',
        ylabel='Change (%)',
        height=300,
        width=CONTENT_WIDTH-100,
        color='#d62728' if monthly_df['MonthlyChange'].iloc[-1] < 0 else '#2ca02c'
    )
    
    # 통계 계산
    current = f"${df['Close'].iloc[-1]:.2f}"
    high = f"${df['Close'].max():.2f}"
    low = f"${df['Close'].min():.2f}"
    average = f"${df['Close'].mean():.2f}"
    ytd_change = f"{(df['Close'].iloc[-1] / df['Close'].iloc[0] - 1) * 100:.2f}%"
    
    # 통계 테이블 너비 조정
    stats_df = pd.DataFrame({
        'Metric': ['Current Price', 'Highest Price', 'Lowest Price', 'Average Price', 'YTD Change'],
        'Value': [current, high, low, average, ytd_change]
    })
    stats_table = pn.widgets.DataFrame(stats_df, name=f'{symbol} Statistics', height=180, width=CONTENT_WIDTH-50)
    
    # 추가 분석 텍스트
    analysis = f"""
    ### {symbol} Market Analysis
    
    The simulated data shows a {'positive' if float(ytd_change.strip('%')) > 0 else 'negative'} year-to-date performance of {ytd_change}.
    The stock reached its highest point of {high} and its lowest point of {low} during the year.
    
    The monthly chart above shows the volatility pattern throughout the year, with each bar representing 
    the percentage change for that month. This helps to identify seasonal patterns or unusual movements.
    """
    
    analysis_pane = pn.pane.Markdown(analysis, width=CONTENT_WIDTH-50)
    
    # 구글 패널 구성 - CSS 클래스 적용
    stock_panel = pn.Column(
        description_pane,
        plot,
        monthly_plot,
        stats_table,
        analysis_pane,
        css_classes=['stock-panel'],
        width=CONTENT_WIDTH-50
    )
    page2_stock_panels.append(stock_panel)

# 페이지 1 레이아웃 구성 - 1행 2열로 설정하고 전체 너비 조정
page1 = pn.Column(
    header1,
    pn.Row(*page1_stock_panels, width=CONTENT_WIDTH),
    css_classes=['dashboard-content'],
    width=CONTENT_WIDTH
)

# 페이지 2 레이아웃 구성 - 전체 너비 조정
page2 = pn.Column(
    header2,
    *page2_stock_panels,
    css_classes=['dashboard-content'],
    width=CONTENT_WIDTH
)

# 페이지 내비게이션을 위한 탭 생성
tabs = pn.Tabs(
    ('Apple & Microsoft', page1),
    ('Google', page2),
    width=CONTENT_WIDTH
)

# 최종 대시보드 구성 - 전체 컨테이너에 CSS 클래스 적용
dashboard = pn.Column(
    pn.pane.Markdown("# Stock Market Analysis Dashboard", align='center'),
    pn.pane.Markdown("This dashboard provides visualization and analysis of major tech stocks in 2024. Navigate between pages using the tabs below.", align='center'),
    tabs,
    css_classes=['dashboard-container'],
    width=PAGE_WIDTH
)

# 경고를 줄이기 위한 설정
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# 추가 서빙 설정: 뷰포트 설정
# pn.config.sizing_mode = 'stretch_width'

# 대시보드를 HTML로 저장
dashboard.save('stock_dashboard_16_9.html', embed=True, resources='cdn')